# Description
This script is used to obtain the best parameters for training the model based on the available data and the options presented to the tuner.

In [1]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os import getcwd
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
image = np.loadtxt('20210805_Cashews_Beni_training.csv', delimiter=',',skiprows=1)

In [3]:
df = pd.read_csv('20210805_Cashews_Beni_training.csv')
Full_data = df
print(df.shape)
df.head()

(1223164, 209)


,fid,label,row_loc,col_loc,tile,20191115_B01,20191115_B02,20191115_B03,20191115_B04,20191115_B05,...,20200319_B03,20200319_B04,20200319_B05,20200319_B06,20200319_B07,20200319_B08,20200319_B8A,20200319_B09,20200319_B11,20200319_B12
0,805,2,19,36,0,327,390,629,405,969,...,1314,1322,1820,2347,2605,2718,2718,3912,2697,1826
1,567,3,19,37,0,327,410,703,460,1000,...,1326,1360,1822,2480,2740,2802,2802,3912,2676,1801
2,567,3,19,38,0,327,407,658,407,1000,...,1314,1384,1822,2480,2740,2680,2680,3912,2676,1801
3,567,3,19,39,0,304,397,654,417,1028,...,1326,1312,1828,2420,2677,2796,2796,3949,2702,1852
4,567,3,19,40,0,304,404,646,442,1028,...,1294,1390,1828,2420,2677,2632,2632,3949,2702,1852


In [4]:
list_of_column_names = list(df.columns)
  
# displaying the list of column names
print('List of column names : ', 
      list_of_column_names)

List of column names :  ['fid', 'label', 'row_loc', 'col_loc', 'tile', '20191115_B01', '20191115_B02', '20191115_B03', '20191115_B04', '20191115_B05', '20191115_B06', '20191115_B07', '20191115_B08', '20191115_B8A', '20191115_B09', '20191115_B11', '20191115_B12', '20191120_B01', '20191120_B02', '20191120_B03', '20191120_B04', '20191120_B05', '20191120_B06', '20191120_B07', '20191120_B08', '20191120_B8A', '20191120_B09', '20191120_B11', '20191120_B12', '20191130_B01', '20191130_B02', '20191130_B03', '20191130_B04', '20191130_B05', '20191130_B06', '20191130_B07', '20191130_B08', '20191130_B8A', '20191130_B09', '20191130_B11', '20191130_B12', '20191205_B01', '20191205_B02', '20191205_B03', '20191205_B04', '20191205_B05', '20191205_B06', '20191205_B07', '20191205_B08', '20191205_B8A', '20191205_B09', '20191205_B11', '20191205_B12', '20191210_B01', '20191210_B02', '20191210_B03', '20191210_B04', '20191210_B05', '20191210_B06', '20191210_B07', '20191210_B08', '20191210_B8A', '20191210_B09', '

In [4]:
fids = df['fid'].unique

In [5]:
max_value = df.to_numpy().max()
print(max_value)

15200


In [6]:
df = df.sample(frac = 1)

In [7]:
train = df[(df['fid'] > 1) & (df['fid'] <= 1295)]
print ("Train data shape:",train.shape)
validation= df[(df['fid'] > 1295) & (df['fid'] <= 1573)]
print ("Validation data shape:",validation.shape)
test = df[df['fid'] > 1573]
print ("Test data shape:",test.shape)

Train data shape: (1016843, 209)
Validation data shape: (95284, 209)
Test data shape: (110975, 209)


In [8]:
print(max_value)

15200


In [9]:
train.head()

,fid,label,row_loc,col_loc,tile,20191115_B01,20191115_B02,20191115_B03,20191115_B04,20191115_B05,...,20200319_B03,20200319_B04,20200319_B05,20200319_B06,20200319_B07,20200319_B08,20200319_B8A,20200319_B09,20200319_B11,20200319_B12
901201,847,5,859,598,0,335,460,692,614,1150,...,1510,1630,2021,2355,2549,2548,2548,4697,3079,2327
377669,491,1,371,969,0,264,329,670,356,1108,...,1426,1486,1981,2585,2814,2790,2790,4410,2982,2000
851262,1113,3,813,63,0,260,322,483,296,750,...,1238,1302,1683,2268,2500,2418,2418,4224,2544,1737
671451,847,5,645,684,0,330,458,727,596,1181,...,1582,1616,2099,2642,2842,3024,3024,4711,3079,2269
718894,847,5,689,871,0,365,398,640,490,953,...,1334,1358,1895,2517,2780,2780,2780,4469,2827,1930


In [10]:
train_npy = train.to_numpy()
test_npy = test.to_numpy()
val_npy = validation.to_numpy()

In [11]:
#data labels
yTrain = train_npy[:,1].astype(int)
print("The initial training labels are ",np.unique(yTrain))
yTrain = yTrain - 1
print("The edited training labels are ",np.unique(yTrain))
yTest = test_npy[:,1].astype(int)
print("The initial testing labels are ",np.unique(yTest))
yTest = yTest - 1
print("The edited testing labels are ",np.unique(yTest))
yVal = val_npy[:,1].astype(int)
print("The initial validation labels are ",np.unique(yVal))
yVal = yVal - 1
print("The edited validation labels are ",np.unique(yVal))

The initial training labels are  [1 2 3 4 5 6]
The edited training labels are  [0 1 2 3 4 5]
The initial testing labels are  [1 2 3 4 5 6]
The edited testing labels are  [0 1 2 3 4 5]
The initial validation labels are  [1 2 3 4 5 6]
The edited validation labels are  [0 1 2 3 4 5]


In [12]:
#imagedata
xTrain = train_npy[:,5:]
print("xTrain shape: ",xTrain.shape)
xTest = test_npy[:,5:]
print("xTest shape: ",xTest.shape)
xVal = val_npy[:,5:]
print("xVal shape: ",xVal.shape)
print("yTest shape: ",yTest.shape)
print("yTrain shape: ",yTrain.shape)
print("yVal shape: ",yVal.shape)
print("Maximum_value: ",max_value)

xTrain shape:  (1016843, 204)
xTest shape:  (110975, 204)
xVal shape:  (95284, 204)
yTest shape:  (110975,)
yTrain shape:  (1016843,)
yVal shape:  (95284,)
Maximum_value:  15200


In [13]:
# Normalise the data
xTrain = xTrain / max_value
xTest = xTest / max_value

# Reshape the data
xTrain = np.reshape(xTrain,(1016843,17,12))
xTest = np.reshape(xTest,(110975,17,12))
# xTrain = np.expand_dims(xTrain,axis =-1)
# xTest = np.expand_dims(xTest,axis =-1)


# Print the shape of reshaped data
print("xTrain:",xTrain.shape)
print("xTest:",xTest.shape)

xTrain: (1016843, 17, 12)
xTest: (110975, 17, 12)


In [14]:
import keras_tuner as kt
from tensorflow import keras
from keras_tuner import RandomSearch

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

In [63]:
#Keras tuner
def build_model(hp):
    # create model object
    model = keras.Sequential([
    #adding first convolutional layer    
    keras.layers.Conv1D(
        #adding filter 
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=hp.Choice('conv_1_kernel', values = [2,7]),
        #activation function
        activation='relu',
        input_shape=(17,12)),
#     adding second convolutional layer 
    keras.layers.Conv1D(
        #adding filter 
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        #adding filter size or kernel size
        kernel_size=hp.Choice('conv_2_kernel', values = [2,7]),
        #activation function
        activation='relu'
    ),
        # adding dense layer 
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
        
    #adding a maxpool layer
    keras.layers.MaxPooling1D(),    
    # adding flatten layer    
    keras.layers.Flatten(),
       
    # output layer    
    keras.layers.Dense(6, activation='softmax')
    ])
    #compilation of model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model
        

In [64]:
tuner = RandomSearch(build_model,
                    objective = 'val_accuracy',
                    max_trials = 5,
                    directory = 'cashews_20210803',
                    project_name = 'Cashew prediction using sentinel 2 images')
# search best parameter
tuner.search(xTrain,yTrain,epochs= 4,validation_data=(xTest,yTest))

Trial 1 Complete [00h 05m 24s]
val_accuracy: 0.09613867849111557

Best val_accuracy So Far: 0.09613867849111557
Total elapsed time: 00h 05m 24s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |48                |80                
conv_1_kernel     |2                 |2                 
conv_2_filter     |48                |48                
conv_2_kernel     |2                 |2                 
dense_1_units     |80                |48                
learning_rate     |0.01              |0.01              

Epoch 1/4
23127/23127 [==============================] - 80s 3ms/step - loss: 1.2785 - accuracy: 0.5047 - val_loss: 2.2063 - val_accuracy: 0.0961
Epoch 2/4
19320/23127 [========================>.....] - ETA: 11s - loss: 1.2775 - accuracy: 0.5048

KeyboardInterrupt: 

In [37]:
model=tuner.get_best_models(num_models=1)[0]
#summary of best model
model.summary()

IndexError: list index out of range

In [16]:
model.fit(xTest,yTest,
          epochs=30,
          validation_split=0.1,
          initial_epoch=3)

Epoch 4/30
7592/7592 [==============================] - 88s 12ms/step - loss: 1.1551 - accuracy: 0.5160 - val_loss: 1.1638 - val_accuracy: 0.5137
Epoch 5/30
7592/7592 [==============================] - 88s 12ms/step - loss: 1.1503 - accuracy: 0.5186 - val_loss: 1.1703 - val_accuracy: 0.5102
Epoch 6/30
7592/7592 [==============================] - 84s 11ms/step - loss: 1.1464 - accuracy: 0.5189 - val_loss: 1.1608 - val_accuracy: 0.5115
Epoch 7/30
7592/7592 [==============================] - 84s 11ms/step - loss: 1.1420 - accuracy: 0.5214 - val_loss: 1.1516 - val_accuracy: 0.5199
Epoch 8/30
7592/7592 [==============================] - 85s 11ms/step - loss: 1.1389 - accuracy: 0.5226 - val_loss: 1.1519 - val_accuracy: 0.5164
Epoch 9/30
7592/7592 [==============================] - 84s 11ms/step - loss: 1.1353 - accuracy: 0.5235 - val_loss: 1.1466 - val_accuracy: 0.5208
Epoch 10/30
7592/7592 [==============================] - 83s 11ms/step - loss: 1.1317 - accuracy: 0.5255 - val_loss: 1.1686 

In [13]:
file_name = 'CNN_20210723'
model.save('file_name')

INFO:tensorflow:Assets written to: file_name/assets


In [15]:
tuner.results_summary()

Results summary
Results in cashews_20210723/Cashew prediction using sentinel 2 images
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 112
learning_rate: 0.001
Score: 0.4984440207481384
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 48
learning_rate: 0.001
Score: 0.4949875473976135
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 5
dense_1_units: 64
learning_rate: 0.001
Score: 0.49479490518569946
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 48
learning_rate: 0.01
Score: 0.4330821633338928
Trial summary
Hyperparameters:
conv_1_filter: 128
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 3
dense_1_units: 80
learning_rate: 0.01
Score: 0.4330821633338928
